First we import the library, this are the library we use

*note: aiogram is telegram bot framework

In [ ]:
import logging
import numpy
import pandas
import requests as req

import aiogram.utils.markdown as md
from aiogram.types import message
from aiogram import Bot, Dispatcher, executor, types
from aiogram.contrib.fsm_storage.memory import MemoryStorage
from aiogram.dispatcher import FSMContext
from aiogram.dispatcher.filters import Text
from aiogram.dispatcher.filters.state import State, StatesGroup
from aiogram.types import ParseMode
from aiogram.utils import executor

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

And then we create sastrawi stemmer object and read response-sentence csv with pandas

In [ ]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()
rsp_list = pandas.read_csv("response.csv")
snt_list = pandas.read_csv("sentence.csv")

## === Main echo functions ===
We're gonna skip the aiogram code and just went straight to main text and data proccessing

So basically the main code to detect user text input is like this

In [ ]:
@dp.message_handler()
async def echo(message: types.Message):
    bot_respon = [] # to save responses
    detected_intent = [] # to save intents
    pesan = message.text.lower() # user input
    kata = pesan.split() # user words (splitted)


    get_responses(kata, detected_intent, bot_respon)      
    bot_respon += get_covid_info( stemmer.stem(synonymize(pesan)) )

    # iterate the bot_respon (if not null)
    for item in bot_respon:
        if item is not None:          
            await message.answer(item) # send the response to user

There's a bunch of made function, so lets head back and see the functions code!

First we make a function to get responses

In [ ]:
def get_responses(words, intents, current_responses):
  for word in words: # iterate user input
      for sentence in snt_list.itertuples(): # convert senctence list and iterate the sentences

          # if stemmed input match the sentence in csv and same intent is not exist
          if stemmer.stem(sentence.Sentence) == stemmer.stem(word) and sentence.Intent not in intents:
            # add both response and intent
            current_responses = add_respon(current_responses, sentence.Intent)            
            intents += [ sentence.Intent ]

Now this is the add_respon function

In [ ]:

def add_respon(responses, key):
  # search response that match the key (intent)
  getResponse = rsp_list[rsp_list["Intent"] == key].to_records(index=False)

  #if not empty the add the response
  if not empty(getResponse):
    responses += [ random.choice(getResponse)[1] ]

  return responses

Empty function's basically just check if array lenght is not 0

In [ ]:
def empty(lists):
  if len(lists) == 0:
    return True

  return False

And we also have synonymize to replace the same intent words into one same word

In [ ]:
def synonymize(words):
  sinonim_list = syn_list.to_records(index=False)
  result = words.lower()

  for sinonims in sinonim_list:
    for item in str(sinonims[1]).split(','):
      result = result.replace(item, f"{sinonims[0]} ", 1)    

  return result

## === Covid Info ===

Next is get_covid_info functions

This function goals is to predict user requested topic of covid related information

In [ ]:
def get_covid_info(user_input):
  response = None
  article = open("article.txt", "r")

  paragraph = article.read().split("\n\n\n")
  tokens = [ stemmer.stem(sw_remover.remove(synonymize(token))) for token in paragraph ]

  tokens.append(user_input)
  vectorized = CountVectorizer().fit_transform(tokens)
  similarity = cosine_similarity(vectorized[-1], vectorized)
  similarity_list = similarity.flatten()
  index = get_similarity_index(similarity_list)
  index = index[1:]

  for x in range(len(index)):
    if similarity_list[index[x]] > 0.2:
      response = f"{paragraph[index[x]]}\n\n"
      break

  tokens.remove(user_input)

  return [ response ]

Let's see the detail what this functions do!

First define null response var and read the article text

In [ ]:
response = None
article = open("article.txt", "r")

Next we split the article into a paragraph

And then we make tokens (a splited stemmed and simplified text) from the paragraph

In [ ]:
paragraph = article.read().split("\n\n\n")
tokens = [ stemmer.stem(sw_remover.remove(synonymize(token))) for token in paragraph ]

After that we add the user input to the tokens and turn it into numpy vector, get the most similarity and turn it into flat list

In [ ]:
tokens.append(user_input)
vectorized = CountVectorizer().fit_transform(tokens)
similarity = cosine_similarity(vectorized[-1], vectorized)
similarity_list = similarity.flatten()

But the problem is the shape of list is not quite right to iterate, so we gonna rotate the list index

In [ ]:
def get_similarity_index(lists):
  lens = len(lists)
  list_index = list(range(0, lens))

  for x in range(lens):
    for y in range(lens):
      if lists[list_index[x]] > lists[list_index][y]:
        swap = list_index[x]
        list_index[x] = list_index[y]
        list_index[y] = swap

  return list_index

Get the index

In [ ]:
index = get_similarity_index(similarity_list)
index = index[1:]

Then iterate the index of similarity list and compare the similarity value with your desired accuracy (we set the accuracy to 0.2)

And lastly return the response (the matched paragraph)

In [ ]:
for x in range(len(index)):
  if similarity_list[index[x]] > 0.2:
    response = f"{paragraph[index[x]]}\n\n"
    break

tokens.remove(user_input)

return [ response ]

## === Predict covid-19 case ===

This is a function to display and predict covid-19 case

First defined what time is now and set the timezone.. also define some var to use later!

In [ ]:
now = datetime.datetime.now(pytz.timezone('Asia/Jakarta'))

head = []
data = []
start = ''

Second step: iterate from 1 - 8

subtract the "now" var for every iterable, get formatted time string, and do a request at "apicovid19indonesia" with the defined formatted time

save the last iterate to "start" variable as a formatted string (now var contains the start date)

add formatted string as a head for every iterate

and lastly add the cases total as the data

In [ ]:
for x in range(1, 8):
  timey = now - datetime.timedelta(days=x)
  d = timey.strftime('%d')
  m = timey.strftime('%B')
  y = timey.strftime('%Y')

  get_covid = requests.get(f"https://apicovid19indonesia-v2.vercel.app/api/indonesia/provinsi/harian?year={y}&date={d}&month={m}")
  covid_info = get_covid.json() 

  if x == 7:
    start = timey.strftime('%d-%b')

  head += [ timey.strftime('%d-%b') ]
  data += [ covid_info['data'][0]['cur_total'] ]

create a pandas dataframe with the predefined head and data

sum all the cases, and also create a numpy array of range from start date and last date

then reshape the cases as numpy array

In [ ]:
confirmed = pandas.DataFrame([data], columns = head)
dates = confirmed.keys()
cases = []

for i in dates:
    cases.append(confirmed[i].sum())

days_since = numpy.array([i for i in range(len(dates))]).reshape(-1, 1)
cases = numpy.array(cases).reshape(-1, 1)

define how many prediction days you want 

create a numpy array of the future date forecast

also define a var for train and test data

split the data to predefined var

In [ ]:
days_in_future = 3
future_forcast = numpy.array([i for i in range(len(dates)+days_in_future)]).reshape(-1, 1)

start_date = datetime.datetime.strptime(start, '%d-%b')
future_forcast_dates = []
cases_date = []
for i in range(len(future_forcast)):
future_forcast_dates.append((start_date + datetime.timedelta(days=i)).strftime('%d-%b'))

X_train_confirmed, X_test_confirmed, y_train_confirmed, y_test_confirmed = train_test_split(days_since, cases, test_size=0.15, shuffle=False)

define a variable for svm predictions

In [ ]:
kernel = ['poly', 'sigmoid', 'rbf']
c = [0.01, 0.1, 1, 10]
gamma = [0.01, 0.1, 1]
epsilon = [0.01, 0.1, 1]
shrinking = [True, False]
svm_grid = {'kernel': kernel, 'C': c, 'gamma' : gamma, 'epsilon': epsilon, 'shrinking' : shrinking}

create a vsm prediction object and create a predictions with the train data

collect the best prediction and finally predict the future covid-19 cases

In [ ]:
svm = SVR()
svm_search = RandomizedSearchCV(svm, svm_grid, scoring='neg_mean_squared_error', cv=3, return_train_score=True, n_jobs=-1, n_iter=40, verbose=1)
svm_search.fit(X_train_confirmed, y_train_confirmed.ravel())

svm_search.best_params_

svm_confirmed = svm_search.best_estimator_
svm_pred = svm_confirmed.predict(future_forcast)

since we use telegram as our wrapper, we cant use matplotlib to display the chart

instead we use __image-charts.com__ web embed image API service's

first we need to reshape the data

In [ ]:
x = ''
y = ''
for i in cases:
    x += f'{round(i[0])},'
for i in svm_pred:
    y += f'{round(i)},'
x = x.rstrip(',')
y = y.rstrip(',')

and then we can output it as a url and the image will displayed to telegram as a embed image

In [ ]:
return f"https://image-charts.com/chart?cht=lc&chd=a:|{ x }|{ y }&chdl=Prediksi|Positif&chxl=0:|{ '|'.join(future_forcast_dates) }|1:||1000|2500|5000|&chs=900x500&chco=3072F3,ff0000&chdlp=t&chls=2,4,1&chm=s,000000,0,-1,5|s,000000,1,-1,5&chxt=x,y"